# Mask R-CNN - Inspect Ballon Trained Model

Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
import mrcnn.model as modellib
from mrcnn.model import log

from samples.japan_roof import japan_roof
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to Ballon trained weights
# You can download this file from the Releases page
# https://github.com/matterport/Mask_RCNN/releases

# ckpt_number = 9245
ckpt_number = 10699
BALLON_WEIGHTS_PATH = "../../logs/pascalvoc20200629T0141/mask_rcnn_pascalvoc_" + str(ckpt_number) + ".h5"  # TODO: update this path

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

## Configurations

In [2]:
config = japan_roof.PascalVOCConfig()
JAPAN_ROOF_DIR = os.path.join(ROOT_DIR, "/root/japan_roof_dataset_2000_solarpanel")
print(JAPAN_ROOF_DIR)

/root/japan_roof_dataset_2000_solarpanel


In [3]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.8
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                22
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Notebook Preferences

In [4]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [5]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Validation Dataset

In [6]:
# Load validation dataset
dataset = japan_roof.PascalVOCDataset()
dataset.load_pascalvoc(JAPAN_ROOF_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

/root/japan_roof_dataset_2000_solarpanel/val/json/[0,11](135.432096E,34.828154N)_center_(135.431664E,34.828506N)min_(135.43252800000002E,34.827802N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,11](135.432096E,34.828154N)_center_(135.431664E,34.828506N)min_(135.43252800000002E,34.827802N)_max_zoom_20_size_640x640.png
/root/japan_roof_dataset_2000_solarpanel/val/json/[1,4](135.514130E,34.581710N)_center_(135.513698E,34.582062N)min_(135.514562E,34.581358N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[1,4](135.514130E,34.581710N)_center_(135.513698E,34.582062N)min_(135.514562E,34.581358N)_max_zoom_20_size_640x640.png
/root/japan_roof_dataset_2000_solarpanel/val/json/[0,4](135.426496E,34.828154N)_center_(135.426064E,34.828506N)min_(135.426928E,34.827802N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,4](135.426496E,34.828154N)_center_(135.426064E,34.828506N)min_(135.426928E,34.

/root/japan_roof_dataset_2000_solarpanel/val/json/[0,3](135.425696E,34.828154N)_center_(135.425264E,34.828506N)min_(135.426128E,34.827802N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,3](135.425696E,34.828154N)_center_(135.425264E,34.828506N)min_(135.426128E,34.827802N)_max_zoom_20_size_640x640.png
/root/japan_roof_dataset_2000_solarpanel/val/json/[0,10](135.563362E,34.640033N)_center_(135.56293000000002E,34.640385N)min_(135.56379400000003E,34.639681N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,10](135.563362E,34.640033N)_center_(135.56293000000002E,34.640385N)min_(135.56379400000003E,34.639681N)_max_zoom_20_size_640x640.png
/root/japan_roof_dataset_2000_solarpanel/val/json/[0,9](135.430496E,34.828154N)_center_(135.43006400000002E,34.828506N)min_(135.43092800000002E,34.827802N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,9](135.430496E,34.828154N)_center_(135.4300640

/root/japan_roof_dataset_2000_solarpanel/val/json/[0,9](135.422567E,34.789354N)_center_(135.422135E,34.789706N)min_(135.422999E,34.789002N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,9](135.422567E,34.789354N)_center_(135.422135E,34.789706N)min_(135.422999E,34.789002N)_max_zoom_20_size_640x640.png
/root/japan_roof_dataset_2000_solarpanel/val/json/[0,0](135.510930E,34.582310N)_center_(135.510498E,34.582662N)min_(135.51136200000002E,34.581958N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,0](135.510930E,34.582310N)_center_(135.510498E,34.582662N)min_(135.51136200000002E,34.581958N)_max_zoom_20_size_640x640.png
/root/japan_roof_dataset_2000_solarpanel/val/json/[0,3](135.557762E,34.640033N)_center_(135.55733E,34.640385N)min_(135.55819400000001E,34.639681N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,3](135.557762E,34.640033N)_center_(135.55733E,34.640385N)min_(135.5581940

/root/japan_roof_dataset_2000_solarpanel/val/json/[0,0](135.423296E,34.828154N)_center_(135.422864E,34.828506N)min_(135.423728E,34.827802N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,0](135.423296E,34.828154N)_center_(135.422864E,34.828506N)min_(135.423728E,34.827802N)_max_zoom_20_size_640x640.png
/root/japan_roof_dataset_2000_solarpanel/val/json/[0,0](135.415367E,34.789354N)_center_(135.414935E,34.789706N)min_(135.41579900000002E,34.789002N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,0](135.415367E,34.789354N)_center_(135.414935E,34.789706N)min_(135.41579900000002E,34.789002N)_max_zoom_20_size_640x640.png
/root/japan_roof_dataset_2000_solarpanel/val/json/[0,0](136.634788E,36.556610N)_center_(136.634356E,36.556962N)min_(136.63522E,36.556258N)_max_zoom_20_size_640x640.json /root/japan_roof_dataset_2000_solarpanel/val/image/[0,0](136.634788E,36.556610N)_center_(136.634356E,36.556962N)min_(136.63522E,36.5562

## Load Model

In [7]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
# Set path to balloon weights file

# Download file from the Releases page and set its path
# https://github.com/matterport/Mask_RCNN/releases
# weights_path = "/path/to/mask_rcnn_balloon.h5"

# Or, load the last model you trained
weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  /root/Mask_RCNN/logs/pascalvoc20200629T0141/mask_rcnn_pascalvoc_9277.h5
Re-starting from epoch 9277


## Run Detection

In [9]:
import colorsys

def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors


In [10]:
def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c],
                                  image[:, :, c])
    return image

In [12]:
import cv2
    
result_save_dir = "./inference_result_" + str(ckpt_number) + "/"
if not os.path.exists(result_save_dir):
    os.mkdir(result_save_dir)

cnt = 0
for path, dirs, files in os.walk("/root/japan_roof_with_trees/"):
# for path, dirs, files in os.walk("/root/japan_roof_dataset_2000_solarpanel/testtest/"):
    for file in files:
        cnt += 1
        image_path = os.path.join(path, file)
        image = io.imread(image_path)

        # png 이미지에 alpha 채널이 있다면 제거 (640, 640 ,4)  >> (640, 640, 3)
        if image.shape[-1] == 4:
            image = image[..., :3]
        
        results = model.detect([image], verbose=1)
        
        # Display results
        r = results[0]
        
        boxes = r['rois']
        masks = r['masks']
        class_ids = r['class_ids']
        class_names = dataset.class_names
        scores = r['scores']
        show_bbox = True
        show_mask = True
        captions = None
        
        # Number of instances
        N = boxes.shape[0]
        print('number_of_instances : ', N)
        if not N:
            print("\n*** No instances to display *** \n")
        else:
            assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

        # Generate random colors
        colors = random_colors(N)

        # Show area outside image boundaries.
        height, width = image.shape[:2]

        masked_image = image.astype(np.uint32).copy()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        for i in range(N):
            float_color = colors[i]
            color = [int(element * 255) for element in float_color]
            
            # Bounding box
            if not np.any(boxes[i]):
                # Skip this instance. Has no bbox. Likely lost in image cropping.
                continue
            y1, x1, y2, x2 = boxes[i]

            if show_bbox:
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                
                
            # Label
            if not captions:
                class_id = class_ids[i]
                score = scores[i] if scores is not None else None
                label = class_names[class_id]
                caption = "{} {:.3f}".format(label, score) if score else label
            else:
                caption = captions[i]
            cv2.putText(image, caption, (x1, y1 + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            
            # Mask
            mask = masks[:, :, i]
            if show_mask:
                masked_image = apply_mask(image, mask, color)

        cv2.imwrite(result_save_dir + file, masked_image)
        print("Image count : ", cnt)
        
        
        

Processing 1 images
image                    shape: (640, 640, 4)         min:    0.00000  max:  255.00000  uint8


ValueError: operands could not be broadcast together with shapes (1024,1024,4) (3,) 